### Desafio Titanic objetivos:
     1) Entender a base (analise descritiva)
     2)Qual o comportamento das variáveis
     3) Existe relação entre as variáveis 
     4) Quais variáveis tiveram impacto na sobrevivencia (Anova)
     5) Seleção de atributos
     6) O quanto que essas variaveis impactaram (Regressão)
     7) Sugestão/Conclusão
  LISTA PERGUNTAS
     1)O Sexo teve interferencia?
     2)Qual a faixa etária com mais mortes?
       prob de mulher, tal idade ter sobrevivido
     3)O lugar de saída teve impacto?
     4)como foi a distribuição das classes por porto? saiu mais de 1a classe de c?

Bibliotecas

In [1]:
import pandas as pd 
import seaborn as sns
import numpy as np
import scipy.stats as sts
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt 
%matplotlib inline 

Importação da base

In [2]:
base =pd.read_csv("C:/Users/012457631/Documents/Cursos_Resumos/Python/Titanic/train.csv")
base.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Formato da base

In [3]:
base.shape

(891, 12)

Informações da base/verificação de vazios

In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#dropar vazios e as colunas que não agregam informação ['PassengerId','Name','Ticket','Cabin ']
base = base[base['Age'].notna()]

In [6]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  714 non-null    int64  
 1   Survived     714 non-null    int64  
 2   Pclass       714 non-null    int64  
 3   Name         714 non-null    object 
 4   Sex          714 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        714 non-null    int64  
 7   Parch        714 non-null    int64  
 8   Ticket       714 non-null    object 
 9   Fare         714 non-null    float64
 10  Cabin        185 non-null    object 
 11  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 72.5+ KB


Como são muito valores diferentes para idade irei dividir a variável em faixas

In [7]:
#faixas etárias
frequencia, classes = np.histogram(base['Age'], bins = 10)
frequencia, classes

(array([ 54,  46, 177, 169, 118,  70,  45,  24,   9,   2], dtype=int64),
 array([ 0.42 ,  8.378, 16.336, 24.294, 32.252, 40.21 , 48.168, 56.126,
        64.084, 72.042, 80.   ]))

In [8]:
base['Faixa Etária'] = pd.cut(base['Age'], bins=[8,16,24,32,40,64,80], 
                        labels=['0 a 8', "9 a 16", "16 a 24", "24 a 32", "32 a 40",'Mais 64'])


In [9]:
base.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Faixa Etária
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,9 a 16
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,24 a 32
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,16 a 24
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,24 a 32
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24 a 32


Dessa base podemos observar que existem vazios tanto na coluna idade quanto na coluna cabine, a coluna cabine é uma variavel mista que não agrega muito o estudo, já a coluna idade pode ser relevante. 

O objetivo é entender a sobrevivencia, por isso iniciarei a analise investigando esta variavel

In [10]:
#value_counts faz a contagem, normalize=True é um argumento para permitir transformar a contagem em porcentual
print(base["Survived"].value_counts(normalize=True).map("{:.1%}".format))

0    59.4%
1    40.6%
Name: Survived, dtype: object


Analisando graficamente todas as variaveis com a segmentação sobrevivencia

In [13]:
# comparar cada coluna da minha tabela com a coluna de cancelamento
import plotly.express as px
# para cada coluna da lista de colunas que a função .columns retorna
colunas=['Faixa Etária', 'SibSp','Parch','Fare','Embarked ','Sex']
for coluna in colunas:
    grafico = px.histogram(base, x=coluna, color="Survived")
    grafico.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Faixa Etária'] but received: Embarked 

In [ ]:
base[['Age', 'SibSp','Parch','Fare']].describe()

Pelos dados descritivos Fare provavelmente não possui distribuição normal, sua média esta muito distante da mediana. As demais variáveis tem uma diferença menor entre estas medidas.
Abaixo os histogramas para melhor avaliação

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,4))
axes[0].hist(base['Age'])
axes[0].set_title("Age")

axes[1].hist(base['SibSp'])
axes[1].set_title("SibSp")

axes[2].hist(base['Parch'])
axes[2].set_title("Parch")

axes[3].hist(base['Fare'])
axes[3].set_title("Fare");

Apesar de idade lembrar a distribuição normal, ela não é simétrica.
Todas as variáveis possuem assimetria para esquerda, ou seja seus dados estão acumulados em valores menores, na embarcação tivemos a maioria mais jovem, com poucos (ou nenhum parente) e que pagaram o menor valor para viajar.

Análise unidimensional categóricas

Analise descritiva por sexo

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Sex']=="male"].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Sex']=="female"].describe()

Quantidade: Na embarcação tiveram mais homens que mulheres.

Idade: Os homens obtiveram idade média e mediana pouco superior as mulheres, possível descarte na influencia da idade na sobrevivencia

Parentes: O valor para parentes parece estranho pois a maioria viajou sem parentes e estamos a olhar a média, mas pode-se afirmar que as mulheres tiveram mais parentes presentes na embarcação que os homens, provavelmente pois na época era responsabilidade das mulheres cuidar da familia, daqueles considerados mais novos que ela.

Tarifa paga (Fare): Aqui temos um sinalizador interessante, tanto na média quanto na mediana as mulheres pagaram praticamente o dobro para entrar na embarcação.

Analise descritiva por classe

In [ ]:
sns.countplot(x='Pclass',data=base)

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Pclass']==1].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Pclass']==2].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Pclass']==3].describe()

Quantidade: A maioria dos viajantes eram da terceira classe

Idade: Aqui é interessantes observar que a idade reduz enquanto a classe vai da primeira a terceira, temos em média pessoas mais velhas na primeira classe e mais novas na terceira, pode-se supor que a pessoa foi adquirindo bens com o tempo e pode assim pagar a mais pelo seu conforto no navio

Parentes: Em relação aos parentes, pouco se difere entre as classes, principalmente entre a primeira e a segunda, a terceira é a que possui a maior média na quantidade de parentes, históricamente é comum familias com menos renda serem mais numerosas que as mais favorecidas financeiramente

Tarifa paga (Fare): Como esperado a primeira classe pagou passagens mais caras que a segunda e a segunda que a terceira

Analise descritiva por embarcação (onde embarcou)

In [ ]:
sns.countplot(x='Embarked',data=base)

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Embarked']=="S"].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Embarked']=="C"].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Embarked']=="Q"].describe()

Informação Relevante: A embarcação partiu em sua viagem inaugural de Southampton com destino a Nova Iorque em 10 de abril de 1912, no caminho passando em Cherbourg-Octeville, na França, e por Queenstown, na Irlanda

Quantidade:A maioria saiu de Southampton (S) e a entrada foi reduzindo nos demais postos, provavelmente pois a quantidade de vagas foi reduzindo ao longo do percurso

Idade:Não há tanta diferenciação entre as idades.

Parentes:Tiveram mais pessoas com parentes em Southampton, mas isso pode ser explicado por serem maioria no navio, logo quanto maior a quantidade de pessoas, maior a probabilidade de estarem acompanhadas

Tarifa paga (fare):As tarifas mais caras foram pagas em Cherbourg-Octeville, em média mais que o dobro que Southampton e mais que o triplo em Queenstown.

Analise descritiva por sobreviviventes

In [ ]:
boxplot tarifasXportoXclasse

In [ ]:
sns.countplot(x='Survived',data=base)

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Survived']==0].describe()

In [ ]:
base[['Age', 'SibSp','Parch','Fare']][base['Survived']==1].describe()

Quantidade: Como sabemos a maioria não sobreviveu.

Idade: A idade precisa de uma analise mais aprofundada, a mediana em ambos os casos é a mesma que a mediana de idade geral (28 anos)

Parentes: Somente com os dados vistos não é possível concluir o impacto  na sobrevivencia por ter parentes a bordo.

Tarifa paga (Fare): Em média, aqueles que sobreviveram pagaram mais que o dobro dos que não, então o fator financeiro teve sim relevancia na sobrevivencia.

Algumas analises gráficas categóricas com segmentação

In [ ]:
sns.barplot(x='Pclass', y='Survived', hue='Sex', data=base)

#adendo mulheres sobreviveram
#fareXclasseXporto
Como esperado a maioria dos sobreviventes foram mulheres das classes mais altas

In [ ]:
sns.barplot(x='Embarked', y='Survived', hue='Sex', data=base)

Apesar de termos visto que a maioria saiu de Southampton, os que tiveram maior exito de sobrevivencia saíram de Cherbourg-Octeville (C), Cherbourg-Octeville foi o porto que teve em média pessoas pagando mais pela passagem, reforçando assim a hipótese que os que pagaram mais tiveram mais exito em sobreviver.

In [ ]:
sns.boxplot(x='Embarked', y='Fare', hue='Pclass', data=base)

Como Sobrevivencia é um dado categórico, é preciso realizar o teste QUI QUADRADO para decidir se o atributo é relevante(se há diferença significativa) o não para o modelo.

In [ ]:
from scipy.stats import chi2
from scipy.stats import chi2_contingency

In [ ]:
freq_table_Sex = pd.crosstab(base['Survived'], base['Sex'])
_, p, _, _ = chi2_contingency(freq_table_Sex)
p

In [ ]:
freq_table_Pclass = pd.crosstab(base['Survived'], base['Pclass'])
_, p, _, _ = chi2_contingency(freq_table_Pclass)
p

In [ ]:
freq_table_Embarked = pd.crosstab(base['Survived'], base['Embarked'])
_, p, _, _ = chi2_contingency(freq_table_Embarked)
p

Todas as variáveis categóricas são relevantes para o modelo, como são categóricas transformarei em dummies para prosseguimento

In [ ]:
dumies=["Sex", "Embarked","Pclass"]
base_dumies = pd.get_dummies(base[dumies])
base_dumies

In [ ]:
base['Pclass'].replace([1,2,3],['classe1','classe2','classe3'])
dumieclasse=pd.get_dummies(base['Pclass'])
base['classe1'] = dumieclasse[1]
base['classe2'] = dumieclasse[2]
base['classe3'] = dumieclasse[3]
base

In [ ]:
resignificar